In [1]:
'''This function evaluates SED of a stellar cluster using a maximum posteriori 
estimation to produce an average best-fit age value. Inputs are two 1 x 22 
arrays, the first array contains the apparent magnitudes in the following 
spectral filters: mag = [FUV, NUV, U, B, V, R, I, J, H, K, u, g, r, i, z, f300w, 
f336w, f435w, f450w, f555w, f606w, f814w]. The second array, err, contains the 
associated errors within each spectral measurement. If data is unavailable for 
a particular filter then its corresponding array element is set to -99. All 
magnitudes are required to be given in the Vega system except for the FUV and 
NUV filters which are required to be in the AB system. Output is a 1 x 4 array
containing the average best-fit ages using the single-star and binary model sets:
[Average best-fit Single log(age), associated uncertainty, Average best-fit 
Binary log(age), associated uncertainty]. Note that dust extinction is not 
accounted for in this estimation, hence the SED must be corrected for dust 
extinction prior to input into this function. Refer to associated dissertation
for a full descripton of this code.'''

def Agefit(mag , err):
    
    import numpy as np
    import math
    
    #Give error message if inputs are in incorrect format.
    if len(mag) != 22 or len(err) != 22:
        print('Error: Inputs Have Incorrect Format')
        return [0 for i in range(4)]
    
    #Load BPASS v. 2.2 Single and Binary model data sets for imf135_300.
    BmodA = [[] for i in range(13)]
    SmodA = [[] for i in range(13)]
    BmodB = [[] for i in range(13)]
    SmodB = [[] for i in range(13)]

    BmodA[0] = np.genfromtxt('colours-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodA[1] = np.genfromtxt('colours-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodA[2] = np.genfromtxt('colours-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodA[3] = np.genfromtxt('colours-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodA[4] = np.genfromtxt('colours-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodA[5] = np.genfromtxt('colours-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodA[6] = np.genfromtxt('colours-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodA[7] = np.genfromtxt('colours-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodA[8] = np.genfromtxt('colours-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodA[9] = np.genfromtxt('colours-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodA[10] = np.genfromtxt('colours-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodA[11] = np.genfromtxt('colours-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodA[12] = np.genfromtxt('colours-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodA[0] = np.genfromtxt('colours-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodA[1] = np.genfromtxt('colours-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodA[2] = np.genfromtxt('colours-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodA[3] = np.genfromtxt('colours-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodA[4] = np.genfromtxt('colours-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodA[5] = np.genfromtxt('colours-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodA[6] = np.genfromtxt('colours-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodA[7] = np.genfromtxt('colours-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodA[8] = np.genfromtxt('colours-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodA[9] = np.genfromtxt('colours-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodA[10] = np.genfromtxt('colours-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodA[11] = np.genfromtxt('colours-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodA[12] = np.genfromtxt('colours-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #The following sets of data are required for BPASS FUV and NUV data.
    BmodB[0] = np.genfromtxt('ionizing-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodB[1] = np.genfromtxt('ionizing-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodB[2] = np.genfromtxt('ionizing-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodB[3] = np.genfromtxt('ionizing-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodB[4] = np.genfromtxt('ionizing-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodB[5] = np.genfromtxt('ionizing-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodB[6] = np.genfromtxt('ionizing-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodB[7] = np.genfromtxt('ionizing-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodB[8] = np.genfromtxt('ionizing-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodB[9] = np.genfromtxt('ionizing-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodB[10] = np.genfromtxt('ionizing-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodB[11] = np.genfromtxt('ionizing-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodB[12] = np.genfromtxt('ionizing-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodB[0] = np.genfromtxt('ionizing-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodB[1] = np.genfromtxt('ionizing-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodB[2] = np.genfromtxt('ionizing-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodB[3] = np.genfromtxt('ionizing-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodB[4] = np.genfromtxt('ionizing-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodB[5] = np.genfromtxt('ionizing-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodB[6] = np.genfromtxt('ionizing-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodB[7] = np.genfromtxt('ionizing-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodB[8] = np.genfromtxt('ionizing-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodB[9] = np.genfromtxt('ionizing-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodB[10] = np.genfromtxt('ionizing-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodB[11] = np.genfromtxt('ionizing-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodB[12] = np.genfromtxt('ionizing-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #Convert ionizing photon production rates into AB magnitudes for FUV and NUV data.
    Smodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(51):
            for k in range(23):
                if k == 0:
                    Smodns[i][j][k] = SmodB[i][j][0]
                    Bmodns[i][j][k] = BmodB[i][j][0]
                elif k == 1:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                elif k == 2:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                else:
                    Smodns[i][j][k] = SmodA[i][j][k - 1]
                    Bmodns[i][j][k] = BmodA[i][j][k - 1]
    
    Smod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(23):
            for k in range(51):
                if j == 0:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                elif (6 + 0.1*k) > 8.5 and k != 50:
                    Smod[i][k][j] = (Smodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                    Bmod[i][k][j] = (Bmodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                else:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
    
    #Store number of spectral measurements from input array.
    count = 0
    for i in range(len(mag)):
        if mag[i] != -99:
            count = count + 1
            
    count1 = 0
    #Observed magnitude and error array
    obs = [0 for i in range(count)]
    obserr = [0 for i in range(count)]
    
    #Corresponding index for model data
    modindex = [0 for i in range(count)]
    
    for i in range(len(mag)):
        if mag[i] != -99:
            obs[i - count1] = mag[i]   
            obserr[i - count1] = err[i]
            modindex[i - count1] = i + 1
        else:
            count1 = count1 + 1
            
    #Find observed V magnitude.
    Vobs = -99
    
    for i in range(count):
        if modindex[i] == 5:
            Vobs = obs[i]
    
    #Return error if no Vobs is found in measurements.
    if (Vobs == -99):
        print('Error: No Vmag')
        return [0 for i in range(4)]
    
    
    ##### AGE ESTIMATES #####
    
    #Set up lists which store the bin widths.
    binwidth = [0 for x in range(51)]
    #Get age binwidths from models.
    for i in range(51):
        if i == 0:
            binwidth[i] = 10**6.05
        else:
            binwidth[i] = (10**(Smod[0][i][0]+0.05) - 10**(Smod[0][i][0]-0.05))
            
    #Create lists to store all liklihoods over all model ages and metallicities.
    SPtotal = [1 for x in range(51*13)]
    BPtotal = [1 for x in range(51*13)]
    
    #Numerators and denominators for average best-fit values.
    Snumage = 0
    Sdenage = 0
    Bnumage = 0
    Bdenage = 0
    
    #Loop through all 51 model ages and 13 model metallicities.
    for j in range(13):
        for k in range(51):
            
            #Magnitude difference between observed and model data.
            Sdiff = Smod[j][k][5] - Vobs
            Bdiff = Bmod[j][k][5] - Vobs
            
            
            #Create lists containing model data for this specific age and metallicity.
            Smodel = [0 for i in range(count)]
            Bmodel = [0 for i in range(count)]
            
            Smodelerr = [0 for i in range(count)]
            Bmodelerr = [0 for i in range(count)]
            
            #Scale model data so that V-band magitudes match that of the observed data.
            for i in range(count):
                Smodel[i] = Smod[j][k][modindex[i]] - Sdiff
                Bmodel[i] = Bmod[j][k][modindex[i]] - Bdiff
                
                #Calculate errors in Sin and Bin models.
                if k == 0:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                elif k == 50:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k - 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k - 1][modindex[i]])/2
                else:
                    Smodelerr[i] = abs(Smod[j][k - 1][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k - 1][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                
            '''Create list for relative liklihoods of model and observed data points matching (count - 1 as we do not use the 
            data point corresponding to V)'''
            Slike = [0 for i in range(count - 1)]
            Blike = [0 for i in range(count - 1)]
            
            #Switch on for V-band magnitude in order to skip it in the evaluation.
            switch = 0
            for i in range(count):
                if modindex[i] == 5:
                    switch = 1
                else:
                    
                    #Total errors (observerd + model) for Sin and Bin likelihood.
                    ser = math.sqrt(Smodelerr[i]**2 + obserr[i]**2)
                    ber = math.sqrt(Bmodelerr[i]**2 + obserr[i]**2)
                    
                    #Likelihood that Sin/Bin filter matching corrsponding observed value.
                    Slike[i - switch] = math.exp(-(Smodel[i] - obs[i])**2/(2*ser**2))
                    Blike[i - switch] = math.exp(-(Bmodel[i] - obs[i])**2/(2*ber**2))
                    
                    
            #Add probability of cluster surviving at a given age.
            Page = 10**((6-Bmod[j][k][0])*0.1)
            
            #Store total liklihood of model of particular age and Z matching data.
            for i in range(count - 1):
                SPtotal[51*j + k] = SPtotal[51*j + k]*Slike[i]
                BPtotal[51*j + k] = BPtotal[51*j + k]*Blike[i]
                
            SPtotal[51*j + k] = SPtotal[51*j + k]*Page
            BPtotal[51*j + k] = BPtotal[51*j + k]*Page
            
            #Estimate average age over all metallicities and ages.
            #Do not exceed the age of the universe in age estimates.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                #Sin Numerator
                Snumage = Snumage + binwidth[k]*SPtotal[51*j + k]*Smod[j][k][0]
                #Sin Denominator
                Sdenage = Sdenage + binwidth[k]*SPtotal[51*j + k]
                
                #Bin Numerator
                Bnumage = Bnumage + binwidth[k]*BPtotal[51*j + k]*Bmod[j][k][0]
                #Bin Denominator
                Bdenage = Bdenage + binwidth[k]*BPtotal[51*j + k]
            
    #Avoid dividing by zero if models show poor match to data.
    if (Sdenage == 0) or (Bdenage == 0):
        return [0 for i in range(4)]
    
    # Average best-fit log(age/yrs) for Sin and Bin model sets.
    #Numerator/Denominator
    Sage = Snumage/Sdenage
    Bage = Bnumage/Bdenage
    
    ##### Error in Evaluations: #####
    
    Snumage = 0
    Sdenage = 0
    Bnumage = 0
    Bdenage = 0
    
    for j in range(13):
        for k in range(51):
            
            #Do not exceed the age of the universe in age estimates
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                Snumage = Snumage + binwidth[k]*SPtotal[51*j + k]*(Smod[j][k][0] - Sage)**2
                Sdenage = Sdenage + binwidth[k]*SPtotal[51*j + k]
                
                Bnumage = Bnumage + binwidth[k]*BPtotal[51*j + k]*(Bmod[j][k][0] - Bage)**2
                Bdenage = Bdenage + binwidth[k]*BPtotal[51*j + k]
            
    #Avoid dividing by zero if models show poor match to data.
    if (Sdenage == 0) or (Bdenage == 0):
        return [0 for i in range(4)]
    
    #error in <log(age/yrs)>
    eSage = Snumage/Sdenage
    eBage = Bnumage/Bdenage
    
    #Return 1 x 4 output array containing estimates: 
    #[Average best-fit Single log(age), associated uncertainty, Average best-fit Binary log(age), associated uncertainty]
    return[Sage,eSage,Bage,eBage]

In [2]:
'''This function evaluates SED of a stellar cluster using a maximum likelihood 
estimation to produce an average best-fit Z value. Inputs are two 1 x 22 
arrays, the first array contains the apparent magnitudes in the following 
spectral filters: mag = [FUV, NUV, U, B, V, R, I, J, H, K, u, g, r, i, z, f300w, 
f336w, f435w, f450w, f555w, f606w, f814w]. The second array, err, contains the 
associated errors within each spectral measurement. If data is unavailable for 
a particular filter then its corresponding array element is set to -99. All 
magnitudes are required to be given in the Vega system except for the FUV and 
NUV filters which are required to be in the AB system. Output is a 1 x 4 array
containing the average best-fit Z values using the single-star and binary model sets:
[Average best-fit Single log(Z), associated uncertainty, Average best-fit 
Binary log(Z), associated uncertainty]. Note that dust extinction is not 
accounted for in this estimation, hence the SED must be corrected for dust 
extinction prior to input into this function. Refer to associated dissertation
for a full descripton of this code.'''

def Zfit(mag , err):
    
    import numpy as np
    import math
    
    #Give error message if inputs are in incorrect format.
    if len(mag) != 22 or len(err) != 22:
        print('Error: Inputs Have Incorrect Format')
        return [0 for i in range(4)]
    
    #Load BPASS v. 2.2 Single and Binary model data sets for imf135_300.
    BmodA = [[] for i in range(13)]
    SmodA = [[] for i in range(13)]
    BmodB = [[] for i in range(13)]
    SmodB = [[] for i in range(13)]

    BmodA[0] = np.genfromtxt('colours-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodA[1] = np.genfromtxt('colours-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodA[2] = np.genfromtxt('colours-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodA[3] = np.genfromtxt('colours-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodA[4] = np.genfromtxt('colours-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodA[5] = np.genfromtxt('colours-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodA[6] = np.genfromtxt('colours-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodA[7] = np.genfromtxt('colours-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodA[8] = np.genfromtxt('colours-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodA[9] = np.genfromtxt('colours-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodA[10] = np.genfromtxt('colours-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodA[11] = np.genfromtxt('colours-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodA[12] = np.genfromtxt('colours-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodA[0] = np.genfromtxt('colours-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodA[1] = np.genfromtxt('colours-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodA[2] = np.genfromtxt('colours-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodA[3] = np.genfromtxt('colours-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodA[4] = np.genfromtxt('colours-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodA[5] = np.genfromtxt('colours-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodA[6] = np.genfromtxt('colours-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodA[7] = np.genfromtxt('colours-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodA[8] = np.genfromtxt('colours-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodA[9] = np.genfromtxt('colours-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodA[10] = np.genfromtxt('colours-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodA[11] = np.genfromtxt('colours-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodA[12] = np.genfromtxt('colours-sin-imf135_300.z040.dat', delimiter = '  ')
    
    BmodB[0] = np.genfromtxt('ionizing-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodB[1] = np.genfromtxt('ionizing-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodB[2] = np.genfromtxt('ionizing-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodB[3] = np.genfromtxt('ionizing-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodB[4] = np.genfromtxt('ionizing-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodB[5] = np.genfromtxt('ionizing-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodB[6] = np.genfromtxt('ionizing-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodB[7] = np.genfromtxt('ionizing-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodB[8] = np.genfromtxt('ionizing-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodB[9] = np.genfromtxt('ionizing-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodB[10] = np.genfromtxt('ionizing-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodB[11] = np.genfromtxt('ionizing-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodB[12] = np.genfromtxt('ionizing-bin-imf135_300.z040.dat', delimiter = '  ')

    #The following sets of data are required for BPASS FUV and NUV data.
    SmodB[0] = np.genfromtxt('ionizing-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodB[1] = np.genfromtxt('ionizing-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodB[2] = np.genfromtxt('ionizing-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodB[3] = np.genfromtxt('ionizing-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodB[4] = np.genfromtxt('ionizing-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodB[5] = np.genfromtxt('ionizing-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodB[6] = np.genfromtxt('ionizing-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodB[7] = np.genfromtxt('ionizing-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodB[8] = np.genfromtxt('ionizing-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodB[9] = np.genfromtxt('ionizing-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodB[10] = np.genfromtxt('ionizing-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodB[11] = np.genfromtxt('ionizing-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodB[12] = np.genfromtxt('ionizing-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #Convert ionizing photon production rates into AB magnitudes for FUV and NUV data.
    Smodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(51):
            for k in range(23):
                if k == 0:
                    Smodns[i][j][k] = SmodB[i][j][0]
                    Bmodns[i][j][k] = BmodB[i][j][0]
                elif k == 1:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                elif k == 2:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                else:
                    Smodns[i][j][k] = SmodA[i][j][k - 1]
                    Bmodns[i][j][k] = BmodA[i][j][k - 1]
    
    Smod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(23):
            for k in range(51):
                if j == 0:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                elif (6 + 0.1*k) > 8.5 and k != 50:
                    Smod[i][k][j] = (Smodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                    Bmod[i][k][j] = (Bmodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                else:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                    
    #Store number of spectral measurements from input array.                
    count = 0
    for i in range(len(mag)):
        if mag[i] != -99:
            count = count + 1
            
    count1 = 0
    #Observed magnitude and error array
    obs = [0 for i in range(count)]
    obserr = [0 for i in range(count)]
    
    #Corresponding index for model data
    modindex = [0 for i in range(count)]
    
    for i in range(len(mag)):
        if mag[i] != -99:
            obs[i - count1] = mag[i]   
            obserr[i - count1] = err[i]
            modindex[i - count1] = i + 1
        else:
            count1 = count1 + 1
            
    #Find observed V magnitude.
    Vobs = -99
    
    for i in range(count):
        if modindex[i] == 5:
            Vobs = obs[i]
    
    #Return error if no Vobs is found in measurements.
    if (Vobs == -99):
        print('Error: No Vmag')
        return [0 for i in range(4)]
    
    
    ##### Metallicity ESTIMATES #####
    
    #store metallicity values in a list
    z = [0.00001 , 0.0001 , 0.001 , 0.002 , 0.003 , 0.004 , 0.006 , 0.008 , 0.010 , 0.014 , 0.020 , 0.030 , 0.040]
    logz = [0 for i in range(13)]
    for i in range(13):
        logz[i] = math.log10(z[i])
    
    #Metallicity bin weighting factors
    totalzbinwidth = 0.040 + 0.5*(0.040-0.030)
    zbinwidth = [0 for i in range(13)]
    for i in range(13):
        if i == 0:
            zbinwidth[i] = (z[i] + 0.5*(z[i+1] - z[i]))
        elif i == 12:
            zbinwidth[i] = (z[i] - z[i-1])
        else:
            zbinwidth[i] = (0.5*(z[i] - z[i-1]) + 0.5*(z[i+1] - z[i]))
            
    #Create lists to store all liklihoods over all model ages and metallicities.
    SPtotal = [1 for x in range(51*13)]
    BPtotal = [1 for x in range(51*13)]
    
    #Numerators and denominators for average best-fit values.
    Snumz = 0
    Sdenz = 0
    Bnumz = 0
    Bdenz = 0
    
    #Loop through all 51 model ages and 13 model metallicities.
    for j in range(13):
        for k in range(51):
            
            #Magnitude difference between observed and model data.
            Sdiff = Smod[j][k][5] - Vobs
            Bdiff = Bmod[j][k][5] - Vobs
            
            
            #Create lists containing model data for this specific age and metallicity.
            Smodel = [0 for i in range(count)]
            Bmodel = [0 for i in range(count)]
            
            Smodelerr = [0 for i in range(count)]
            Bmodelerr = [0 for i in range(count)]
            
            #Scale model data so that V-band magitudes match that of the observed data.
            for i in range(count):
                Smodel[i] = Smod[j][k][modindex[i]] - Sdiff
                Bmodel[i] = Bmod[j][k][modindex[i]] - Bdiff
                
                #Calculate errors in Sin and Bin models.
                if k == 0:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                elif k == 50:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k - 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k - 1][modindex[i]])/2
                else:
                    Smodelerr[i] = abs(Smod[j][k - 1][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k - 1][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                
            '''Create list for relative liklihoods of model and observed data points matching (count - 1 as we do not use the 
            data point corresponding to V)'''
            Slike = [0 for i in range(count - 1)]
            Blike = [0 for i in range(count - 1)]
            
            #Switch on for V-band magnitude in order to skip it in the evaluation.
            switch = 0
            for i in range(count):
                if modindex[i] == 5:
                    switch = 1
                else:
                    
                    #Total errors (observerd + model) for Sin and Bin likelihood.
                    ser = math.sqrt(Smodelerr[i]**2 + obserr[i]**2)
                    ber = math.sqrt(Bmodelerr[i]**2 + obserr[i]**2)
                    
                    #Likelihood that Sin/Bin filter matching corrsponding observed value.
                    Slike[i - switch] = math.exp(-(Smodel[i] - obs[i])**2/(2*ser**2))
                    Blike[i - switch] = math.exp(-(Bmodel[i] - obs[i])**2/(2*ber**2))
            
            #Store liklihood of model matching data
            for i in range(count - 1):
                SPtotal[51*j + k] = SPtotal[51*j + k]*Slike[i]
                BPtotal[51*j + k] = BPtotal[51*j + k]*Blike[i]
            
            #Estimate average Z over all metallicities and ages.
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                #Sin Numerator
                Snumz = Snumz + zbinwidth[j]*SPtotal[51*j + k]*logz[j]
                #Sin Denominator
                Sdenz = Sdenz + zbinwidth[j]*SPtotal[51*j + k]
                
                #Bin Numerator
                Bnumz = Bnumz + zbinwidth[j]*BPtotal[51*j + k]*logz[j]
                #Bin Denominator
                Bdenz = Bdenz + zbinwidth[j]*BPtotal[51*j + k]
            
    #Avoid dividing by zero if models show poor match to data.
    if (Sdenz == 0) or (Bdenz == 0):
        return [0 for i in range(4)]
    
    # Average best-fit log(Z) for Sin and Bin model sets.
    #Numerator/Denominator
    Sz = Snumz/Sdenz
    Bz = Bnumz/Bdenz
    
    ##### Error in Evaluations: #####
    
    Snumz = 0
    Sdenz = 0
    Bnumz = 0
    Bdenz = 0
    
    for j in range(13):
        for k in range(51):
            
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                Snumz = Snumz + zbinwidth[j]*SPtotal[51*j + k]*(logz[j] - Sz)**2
                Sdenz = Sdenz + zbinwidth[j]*SPtotal[51*j + k]
                
                Bnumz = Bnumz + zbinwidth[j]*BPtotal[51*j + k]*(logz[j] - Bz)**2
                Bdenz = Bdenz + zbinwidth[j]*BPtotal[51*j + k]
            
    #Avoid dividing by zero if models show poor match to data.
    if (Sdenz == 0) or (Bdenz == 0):
        return [0 for i in range(4)]
    
    #error in <log(Z)>
    eSz = Snumz/Sdenz
    eBz = Bnumz/Bdenz
    
    #Return 1 x 4 output array containing estimates: 
    #[Average best-fit Single log(Z), associated uncertainty, Average best-fit Binary log(Z), associated uncertainty]
    return[Sz,eSz,Bz,eBz]

In [3]:
'''This function evaluates SED of a stellar cluster using a maximum posteriori 
estimation to produce an average best-fit V-band extinction value. Inputs are two 1 x 22 
arrays and one float. The first 1 x 22 array contains the apparent magnitudes in the following 
spectral filters: mag = [FUV, NUV, U, B, V, R, I, J, H, K, u, g, r, i, z, f300w, 
f336w, f435w, f450w, f555w, f606w, f814w]. The second array, err, contains the 
associated errors within each spectral measurement. If data is unavailable for 
a particular filter then its corresponding array element is set to -99. All 
magnitudes are required to be given in the Vega system except for the FUV and 
NUV filters which are required to be in the AB system. The third input, Amin, is a float 
specifying the minimum V-band extinction for the object. If no data is available for this 
value then it is set to Amin = 0. Output is a 1 x 4 array containing the average 
best-fit A_V values using the single-star and binary model sets:
[Average best-fit Single A_V, associated uncertainty, Average best-fit 
Binary A_V, associated uncertainty]. Note that dust extinction is not 
accounted for in this estimation, hence the SED must be corrected for dust 
extinction prior to input into this function. Refer to associated dissertation
for a full descripton of this code.'''

def Dustfit(mag , err, Amin):
    
    import numpy as np
    import math
    
    #Give error message if inputs are in incorrect format.
    if len(mag) != 22 or len(err) != 22:
        print('Error: Inputs Have Incorrect Format')
        return [0 for i in range(4)]
    
    #Load BPASS v. 2.2 Single and Binary model data sets for imf135_300.
    BmodA = [[] for i in range(13)]
    SmodA = [[] for i in range(13)]
    BmodB = [[] for i in range(13)]
    SmodB = [[] for i in range(13)]

    BmodA[0] = np.genfromtxt('colours-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodA[1] = np.genfromtxt('colours-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodA[2] = np.genfromtxt('colours-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodA[3] = np.genfromtxt('colours-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodA[4] = np.genfromtxt('colours-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodA[5] = np.genfromtxt('colours-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodA[6] = np.genfromtxt('colours-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodA[7] = np.genfromtxt('colours-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodA[8] = np.genfromtxt('colours-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodA[9] = np.genfromtxt('colours-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodA[10] = np.genfromtxt('colours-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodA[11] = np.genfromtxt('colours-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodA[12] = np.genfromtxt('colours-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodA[0] = np.genfromtxt('colours-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodA[1] = np.genfromtxt('colours-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodA[2] = np.genfromtxt('colours-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodA[3] = np.genfromtxt('colours-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodA[4] = np.genfromtxt('colours-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodA[5] = np.genfromtxt('colours-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodA[6] = np.genfromtxt('colours-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodA[7] = np.genfromtxt('colours-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodA[8] = np.genfromtxt('colours-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodA[9] = np.genfromtxt('colours-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodA[10] = np.genfromtxt('colours-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodA[11] = np.genfromtxt('colours-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodA[12] = np.genfromtxt('colours-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #The following sets of data are required for BPASS FUV and NUV data.
    BmodB[0] = np.genfromtxt('ionizing-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodB[1] = np.genfromtxt('ionizing-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodB[2] = np.genfromtxt('ionizing-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodB[3] = np.genfromtxt('ionizing-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodB[4] = np.genfromtxt('ionizing-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodB[5] = np.genfromtxt('ionizing-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodB[6] = np.genfromtxt('ionizing-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodB[7] = np.genfromtxt('ionizing-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodB[8] = np.genfromtxt('ionizing-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodB[9] = np.genfromtxt('ionizing-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodB[10] = np.genfromtxt('ionizing-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodB[11] = np.genfromtxt('ionizing-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodB[12] = np.genfromtxt('ionizing-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodB[0] = np.genfromtxt('ionizing-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodB[1] = np.genfromtxt('ionizing-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodB[2] = np.genfromtxt('ionizing-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodB[3] = np.genfromtxt('ionizing-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodB[4] = np.genfromtxt('ionizing-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodB[5] = np.genfromtxt('ionizing-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodB[6] = np.genfromtxt('ionizing-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodB[7] = np.genfromtxt('ionizing-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodB[8] = np.genfromtxt('ionizing-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodB[9] = np.genfromtxt('ionizing-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodB[10] = np.genfromtxt('ionizing-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodB[11] = np.genfromtxt('ionizing-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodB[12] = np.genfromtxt('ionizing-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #Convert ionizing photon production rates into AB magnitudes for FUV and NUV data.
    Smodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(51):
            for k in range(23):
                if k == 0:
                    Smodns[i][j][k] = SmodB[i][j][0]
                    Bmodns[i][j][k] = BmodB[i][j][0]
                elif k == 1:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                elif k == 2:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                else:
                    Smodns[i][j][k] = SmodA[i][j][k - 1]
                    Bmodns[i][j][k] = BmodA[i][j][k - 1]
    
    Smod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(23):
            for k in range(51):
                if j == 0:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                elif (6 + 0.1*k) > 8.5 and k != 50:
                    Smod[i][k][j] = (Smodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                    Bmod[i][k][j] = (Bmodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                else:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
     
    #Store number of spectral measurements from input array.
    count = 0
    for i in range(len(mag)):
        if mag[i] != -99:
            count = count + 1
            
    count1 = 0
    #Observed magnitude and error array
    obs = [0 for i in range(count)]
    obserr = [0 for i in range(count)]
    
    #Corresponding index for model
    modindex = [0 for i in range(count)]
    
    for i in range(len(mag)):
        if mag[i] != -99:
            obs[i - count1] = mag[i]   
            obserr[i - count1] = err[i]
            modindex[i - count1] = i + 1
        else:
            count1 = count1 + 1
            
    #Find observed V magnitude.
    Vobs = -99
    
    for i in range(count):
        if modindex[i] == 5:
            Vobs = obs[i]
    
    #Return error if no Vobs is found in measurements.
    if (Vobs == -99):
        print('Error: No Vmag')
        return [0 for i in range(4)]
    
    
    ##### DUST ESTIMATES #####
    
    #Python package containing emperical exinction laws
    import extinction
    
    #Range of possible absolute extinction values, zero point is intergalactic extinction
    A_V = [(Amin + 0.1*i) for i in range(101)]
            
    #Create lists to store all liklihoods over all model ages and metallicities.
    SPtotal = [[[1. for i in range(101)] for j in range(51)] for k in range(13)]
    BPtotal = [[[1. for i in range(101)] for j in range(51)] for k in range(13)]
    
    SnumAv = 0
    SdenAv = 0
    BnumAv = 0
    BdenAv = 0
    
    #Loop through all 51 model ages, 13 model metallicities and 101 possible A_V ( V-band extinction) values.
    for j in range(13):
        for k in range(51):
            for x in range(101):
                
                #Pre-calculated extinction array corresponding to spectral filters for A_V = 1.0.
                ext = [2.6517 , 2.6331 , 
               1.5578462941114875, 1.3093650372780916, 1.0053521386991533, 0.7614405115045431, 0.5929401215210357, 0.2880391530575151, 0.1859713631042207, 0.1155345186186997 ,
               1.5822202835340584, 1.21960961977842, 0.8785056211191205, 0.673375361689415, 0.4882034412598298 , 
               1.8252115 , 1.63853876 , 1.34967994 , 1.26766918 , 1.01183735 , 0.90721259 , 0.59576871]
                
                #find difference minus A_V extinction (as dust is added to the models later on in evaluation),
                #A_V[x] is a scaling factor of the ext array.
                Sdiff = Smod[j][k][5] - Vobs + ext[4]*A_V[x]
                Bdiff = Bmod[j][k][5] - Vobs + ext[4]*A_V[x]
            
            
                #Create lists containing model data for this specific age and metallicity
                Smodel = [0 for i in range(count)]
                Bmodel = [0 for i in range(count)]
            
                Smodelerr = [0 for i in range(count)]
                Bmodelerr = [0 for i in range(count)]
            
                #Scale model data and add dust so that V-band magitudes match that of the observed data.
                for i in range(count):
                    Smodel[i] = Smod[j][k][modindex[i]] - Sdiff + ext[modindex[i] - 1]*A_V[x]
                    Bmodel[i] = Bmod[j][k][modindex[i]] - Bdiff + ext[modindex[i] - 1]*A_V[x]
                
                    #Calculate errors in Sin and Bin models.
                    if k == 0:
                        Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                        Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                    elif k == 50:
                        Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k - 1][modindex[i]])/2
                        Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k - 1][modindex[i]])/2
                    else:
                        Smodelerr[i] = abs(Smod[j][k - 1][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                        Bmodelerr[i] = abs(Bmod[j][k - 1][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                
                '''Create list for relative liklihoods of model and observed data points matching (count - 1 as we do not use the 
                data point corresponding to V)'''
                Slike = [0 for i in range(count - 1)]
                Blike = [0 for i in range(count - 1)]
            
                #Switch on for V-band magnitude in order to skip it in the evaluation.
                switch = 0
                for i in range(count):
                    if modindex[i] == 5:
                        switch = 1
                    else:

                        #Total errors (observerd + model) for Sin and Bin likelihood.
                        ser = math.sqrt(Smodelerr[i]**2 + obserr[i]**2)
                        ber = math.sqrt(Bmodelerr[i]**2 + obserr[i]**2)

                        #Likelihood that Sin/Bin filter matching corrsponding observed value.
                        Slike[i - switch] = math.exp(-(Smodel[i] - obs[i])**2/(2*ser**2))
                        Blike[i - switch] = math.exp(-(Bmodel[i] - obs[i])**2/(2*ber**2))
            
                #Store liklihood of model matching data
                for i in range(count - 1):
                    SPtotal[j][k][x] = SPtotal[j][k][x]*Slike[i]
                    BPtotal[j][k][x] = BPtotal[j][k][x]*Blike[i]

                #Add Prior distrubution to the dust: Exponential decrease so probability of having a high amount of extinction is low.
                Pdust = math.exp(-2*(A_V[x] + Amin))
                
                SPtotal[j][k][x] = SPtotal[j][k][x]*Pdust
                BPtotal[j][k][x] = BPtotal[j][k][x]*Pdust
                
    #Estimate average A_V over all metallicities, ages and extinction range.        
    for j in range(13):
        for k in range(51):
            for x in range(101):
    
                #Exclude model SEDs older than those of the universe in evaluation.
                if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                    #Sin Numerator
                    SnumAv = SnumAv + SPtotal[j][k][x]*A_V[x]
                    #Sin Denominator
                    SdenAv = SdenAv + SPtotal[j][k][x]

                    #Sin Numerator
                    BnumAv = BnumAv + BPtotal[j][k][x]*A_V[x]
                    #Bin Denominator
                    BdenAv = BdenAv + BPtotal[j][k][x]
            
    #Avoid dividing by zero if models show poor match to data.
    if (SdenAv == 0) or (BdenAv == 0):
        return [0 for i in range(4)]
    
    #Single and binary Average Best-Fit A_V
    #Numerator/Denominator
    SAv = SnumAv/SdenAv
    BAv = BnumAv/BdenAv
    
    ##### Error in Evaluations: #####
    
    SnumAv = 0
    SdenAv = 0
    BnumAv = 0
    BdenAv = 0
    
    for j in range(13):
        for k in range(51):
            for x in range(11):
            
                #Exclude model SEDs older than those of the universe in evaluation.
                if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                    SnumAv = SnumAv + SPtotal[j][k][x]*(A_V[x] - SAv)**2
                    SdenAv = SdenAv + SPtotal[j][k][x]

                    BnumAv = BnumAv + BPtotal[j][k][x]*(A_V[x] - BAv)**2
                    BdenAv = BdenAv + BPtotal[j][k][x]
            
    #Avoid dividing by zero if models show poor match to data.
    if (SdenAv == 0) or (BdenAv == 0):
        return [0 for i in range(4)]
    
    #error in <A_V>
    eSAv = SnumAv/SdenAv
    eBAv = BnumAv/BdenAv
    
    #Return 1 x 4 output array containing estimates: 
    #[Average best-fit Single A_V, associated uncertainty, Average best-fit Binary A_V, associated uncertainty]
    return[SAv,eSAv,BAv,eBAv]

In [4]:
'''This function evaluates SED of a stellar cluster using a maximum likelihood 
estimation to produce an average best-fit Mass. Inputs are two 1 x 22 
arrays and a 1 x 2 array. The first 1 x 22 array contains the apparent magnitudes 
in the following spectral filters: mag = [FUV, NUV, U, B, V, R, I, J, H, K, u, g, 
r, i, z, f300w, f336w, f435w, f450w, f555w, f606w, f814w]. The second array, err, 
contains the associated errors within each spectral measurement. If data is unavailable for 
a particular filter then its corresponding array element is set to -99. All 
magnitudes are required to be given in the Vega system except for the FUV and 
NUV filters which are required to be in the AB system. The 1 x 2 array input specifies 
the distance modulus of the stellar cluster and the associated uncertainty, 
dmod = [Distance Modulus, Error in Measurement]. 
Output is a 1 x 4 array containing the average best-fit log(M/M_solar) values 
using the single-star and binary model sets:
[Average best-fit Single log(M/M_solar), associated uncertainty, Average best-fit 
Binary log(M/M_solar), associated uncertainty]. Note that dust extinction is not 
accounted for in this estimation, hence the SED must be corrected for dust 
extinction prior to input into this function. Refer to associated dissertation
for a full descripton of this code.'''

def Massfitv1(mag , err, dmod = None):
    
    import numpy as np
    import math
    
    #Give error message if no value for the distance modulus is given.
    if dmod is None:
        print('Error: No Distance Modulus')
        return[0 for i in range(4)]
    
    #Give error message if inputs are in incorrect format.
    if len(mag) != 22 or len(err) != 22:
        print('Error: Inputs Have Incorrect Format')
        return [0 for i in range(4)]
    
    #Load BPASS v. 2.2 Single and Binary model data sets for imf135_300.
    BmodA = [[] for i in range(13)]
    SmodA = [[] for i in range(13)]
    BmodB = [[] for i in range(13)]
    SmodB = [[] for i in range(13)]

    BmodA[0] = np.genfromtxt('colours-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodA[1] = np.genfromtxt('colours-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodA[2] = np.genfromtxt('colours-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodA[3] = np.genfromtxt('colours-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodA[4] = np.genfromtxt('colours-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodA[5] = np.genfromtxt('colours-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodA[6] = np.genfromtxt('colours-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodA[7] = np.genfromtxt('colours-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodA[8] = np.genfromtxt('colours-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodA[9] = np.genfromtxt('colours-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodA[10] = np.genfromtxt('colours-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodA[11] = np.genfromtxt('colours-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodA[12] = np.genfromtxt('colours-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodA[0] = np.genfromtxt('colours-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodA[1] = np.genfromtxt('colours-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodA[2] = np.genfromtxt('colours-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodA[3] = np.genfromtxt('colours-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodA[4] = np.genfromtxt('colours-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodA[5] = np.genfromtxt('colours-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodA[6] = np.genfromtxt('colours-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodA[7] = np.genfromtxt('colours-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodA[8] = np.genfromtxt('colours-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodA[9] = np.genfromtxt('colours-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodA[10] = np.genfromtxt('colours-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodA[11] = np.genfromtxt('colours-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodA[12] = np.genfromtxt('colours-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #The following sets of data are required for BPASS FUV and NUV data.
    BmodB[0] = np.genfromtxt('ionizing-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodB[1] = np.genfromtxt('ionizing-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodB[2] = np.genfromtxt('ionizing-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodB[3] = np.genfromtxt('ionizing-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodB[4] = np.genfromtxt('ionizing-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodB[5] = np.genfromtxt('ionizing-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodB[6] = np.genfromtxt('ionizing-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodB[7] = np.genfromtxt('ionizing-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodB[8] = np.genfromtxt('ionizing-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodB[9] = np.genfromtxt('ionizing-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodB[10] = np.genfromtxt('ionizing-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodB[11] = np.genfromtxt('ionizing-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodB[12] = np.genfromtxt('ionizing-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodB[0] = np.genfromtxt('ionizing-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodB[1] = np.genfromtxt('ionizing-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodB[2] = np.genfromtxt('ionizing-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodB[3] = np.genfromtxt('ionizing-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodB[4] = np.genfromtxt('ionizing-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodB[5] = np.genfromtxt('ionizing-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodB[6] = np.genfromtxt('ionizing-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodB[7] = np.genfromtxt('ionizing-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodB[8] = np.genfromtxt('ionizing-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodB[9] = np.genfromtxt('ionizing-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodB[10] = np.genfromtxt('ionizing-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodB[11] = np.genfromtxt('ionizing-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodB[12] = np.genfromtxt('ionizing-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #Convert ionizing photon production rates into AB magnitudes for FUV and NUV data.
    Smodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(51):
            for k in range(23):
                if k == 0:
                    Smodns[i][j][k] = SmodB[i][j][0]
                    Bmodns[i][j][k] = BmodB[i][j][0]
                elif k == 1:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                elif k == 2:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                else:
                    Smodns[i][j][k] = SmodA[i][j][k - 1]
                    Bmodns[i][j][k] = BmodA[i][j][k - 1]
    
    Smod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(23):
            for k in range(51):
                if j == 0:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                elif (6 + 0.1*k) > 8.5 and k != 50:
                    Smod[i][k][j] = (Smodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                    Bmod[i][k][j] = (Bmodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                else:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
     
    #Store number of spectral measurements from input array.
    count = 0
    for i in range(len(mag)):
        if mag[i] != -99:
            count = count + 1
            
    count1 = 0
    #Observed magnitude and error array
    obs = [0 for i in range(count)]
    obserr = [0 for i in range(count)]
    
    #Corresponding index for model
    modindex = [0 for i in range(count)]
    
    for i in range(len(mag)):
        if mag[i] != -99:
            obs[i - count1] = mag[i]   
            obserr[i - count1] = err[i]
            modindex[i - count1] = i + 1
        else:
            count1 = count1 + 1
            
    #Find observed V magnitude.
    Vobs = -99
    
    for i in range(count):
        if modindex[i] == 5:
            Vobs = obs[i]
    
    #Return error if no Vobs is found in measurements.
    if (Vobs == -99):
        print('Error: No Vmag')
        return [0 for i in range(4)]
    
    ##### MASS ESTIMATES #####
      
    #Absolute Magnitudes.
    Mabs = [0 for i in range(count)]
    
    #Convert observed data from apparent to absolute magnitudes
    for i in range(count):
        Mabs[i] = obs[i] - dmod[0]

    #Create lists to store all liklihoods over all model ages and metallicities.
    SPtotal = [1 for x in range(51*13)]
    BPtotal = [1 for x in range(51*13)]
    
    #Array to store all possible masses.
    logSmasstotal = [0 for x in range(51*13)]
    logBmasstotal = [0 for x in range(51*13)]

    Snummass = 0
    Sdenmass = 0
    Bnummass = 0
    Bdenmass = 0

    #Loop through all 51 model ages and 13 model metallicities.
    for j in range(13):
        for k in range(51):
            
            #Difference in absolute V-band magnitudes between observed and model data (use this value to find the mass).
            Sdiff = Smod[j][k][5] - (Vobs - dmod[0])
            Bdiff = Bmod[j][k][5] - (Vobs - dmod[0])

            #Create lists containing model data for this specific age and metallicity
            Smodel = [0 for i in range(count)]
            Bmodel = [0 for i in range(count)]
            
            Smodelerr = [0 for i in range(count)]
            Bmodelerr = [0 for i in range(count)]
            
            #Scale model data so that V-band magitudes match that of the observed data.
            for i in range(count):
                Smodel[i] = Smod[j][k][modindex[i]] - Sdiff
                Bmodel[i] = Bmod[j][k][modindex[i]] - Bdiff
                
                #Calculate errors in Sin and Bin models.
                if k == 0:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                elif k == 50:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k - 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k - 1][modindex[i]])/2
                else:
                    Smodelerr[i] = abs(Smod[j][k - 1][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k - 1][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2

            '''Create list for relative liklihoods of model and observed data points matching (count - 1 as we do not use the 
            data point corresponding to V)'''
            Slike = [0 for i in range(count - 1)]
            Blike = [0 for i in range(count - 1)]

            #Switch on for V-band magnitude in order to skip it in the evaluation.
            switch = 0
            for i in range(count):
                if modindex[i] == 5:
                    switch = 1
                else:
                    
                    #Total errors (observerd + model + dist. mod.) for Sin and Bin likelihood.
                    ser = math.sqrt(Smodelerr[i]**2 + obserr[i]**2 + dmod[1]**2)
                    ber = math.sqrt(Bmodelerr[i]**2 + obserr[i]**2 + dmod[1]**2)

                    #Likelihood that Sin/Bin filter matching corrsponding observed value.
                    Slike[i - switch] = math.exp(-(Smodel[i] - Mabs[i])**2/(2*ser**2))
                    Blike[i - switch] = math.exp(-(Bmodel[i] - Mabs[i])**2/(2*ber**2))

            #Store liklihood of model matching data
            for i in range(count - 1):
                SPtotal[51*j + k] = SPtotal[51*j + k]*Slike[i]
                BPtotal[51*j + k] = BPtotal[51*j + k]*Blike[i]

            #Mass based on difference between observed absolute magnitude in V filter and V model
            logSmasstotal[51*j + k] = math.log10(10**6*10**(Sdiff/2.5))
            logBmasstotal[51*j + k] = math.log10(10**6*10**(Bdiff/2.5))

            #Estimate average log(M/M_solar) over all metallicities, ages
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                #Sin Numerator
                Snummass = Snummass + SPtotal[51*j + k]*logSmasstotal[51*j + k]
                #Sin Denominator
                Sdenmass = Sdenmass + SPtotal[51*j + k]

                #Bin Numerator
                Bnummass = Bnummass + BPtotal[51*j + k]*logBmasstotal[51*j + k]
                #Bin Denominator
                Bdenmass = Bdenmass + BPtotal[51*j + k]

    #Avoid dividing by zero if models show poor match to data.
    if (Sdenmass == 0) or (Bdenmass == 0):
        return [0 for i in range(4)]

    #Single and binary Average Best-Fit log(M/M_solar)
    #Numerator/Denominator
    Smass = Snummass/Sdenmass
    Bmass = Bnummass/Bdenmass

    ##### Error in Evaluations: #####
    
    Snummass = 0
    Sdenmass = 0
    Bnummass = 0
    Bdenmass = 0

    for j in range(13):
        for k in range(51):
            
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                Snummass = Snummass + SPtotal[51*j + k]*(logSmasstotal[51*j + k] - Smass)**2
                Sdenmass = Sdenmass + SPtotal[51*j + k]

                Bnummass = Bnummass + BPtotal[51*j + k]*(logBmasstotal[51*j + k] - Bmass)**2
                Bdenmass = Bdenmass + BPtotal[51*j + k]

    #Avoid dividing by zero if models show poor match to data.
    if (Sdenmass == 0) or (Bdenmass == 0):
        return [0 for i in range(4)]
    
    #error in <log(M/M_solar)>
    eSmass = Snummass/Sdenmass
    eBmass = Bnummass/Bdenmass

    #Return 1 x 4 output array containing estimates: 
    #[Average best-fit Single log(M/M_solar), associated uncertainty, Average best-fit Binary log(M/M_solar), associated uncertainty]
    return[Smass,eSmass,Bmass,eBmass]

In [5]:
'''This function evaluates SED of a stellar cluster using a maximum posteriori 
estimation to produce an average best-fit age, metllicity and V-band extinction. Inputs are two 1 x 22 
arrays and one float. The first 1 x 22 array contains the apparent magnitudes in the following 
spectral filters: mag = [FUV, NUV, U, B, V, R, I, J, H, K, u, g, r, i, z, f300w, 
f336w, f435w, f450w, f555w, f606w, f814w]. The second array, err, contains the 
associated errors within each spectral measurement. If data is unavailable for 
a particular filter then its corresponding array element is set to -99. All 
magnitudes are required to be given in the Vega system except for the FUV and 
NUV filters which are required to be in the AB system. The third input, Amin, is a float 
specifying the minimum V-band extinction for the object. If no data is available for this 
value then it is set to Amin = 0. Output is a 1 x 12 array containing the average 
best-fit log(age/yrs), log(Z) and A_V values using the single-star and binary model sets:
[Average best-fit Single log(age/yrs), associated uncertainty, Average best-fit 
Binary log(age/yrs), associated uncertainty, Average best-fit Single log(Z), associated 
uncertainty, Average best-fit Binary log(Z), associated uncertainty, Average best-fit 
Single A_V, associated uncertainty, Average best-fit Binary A_V, associated uncertainty]. 
Refer to associated dissertation for a full descripton of this code.'''

##### SECOND METHOD - DUST INCORPORATED #####

# Age, Z and Av Estimation Code

def Clusterfit(mag , err, Amin = None):
    
    #Default minimum V-band extinction is zero.
    if Amin is None:
        Amin = 0
    
    import numpy as np
    import math
    
    #Give error message if inputs are in incorrect format.
    if len(mag) != 22 or len(err) != 22:
        print('Error: Inputs Have Incorrect Format')
        return [0 for i in range(12)]
    
    #Load BPASS v. 2.2 Single and Binary model data sets for imf135_300.
    BmodA = [[] for i in range(13)]
    SmodA = [[] for i in range(13)]
    BmodB = [[] for i in range(13)]
    SmodB = [[] for i in range(13)]

    BmodA[0] = np.genfromtxt('colours-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodA[1] = np.genfromtxt('colours-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodA[2] = np.genfromtxt('colours-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodA[3] = np.genfromtxt('colours-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodA[4] = np.genfromtxt('colours-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodA[5] = np.genfromtxt('colours-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodA[6] = np.genfromtxt('colours-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodA[7] = np.genfromtxt('colours-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodA[8] = np.genfromtxt('colours-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodA[9] = np.genfromtxt('colours-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodA[10] = np.genfromtxt('colours-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodA[11] = np.genfromtxt('colours-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodA[12] = np.genfromtxt('colours-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodA[0] = np.genfromtxt('colours-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodA[1] = np.genfromtxt('colours-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodA[2] = np.genfromtxt('colours-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodA[3] = np.genfromtxt('colours-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodA[4] = np.genfromtxt('colours-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodA[5] = np.genfromtxt('colours-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodA[6] = np.genfromtxt('colours-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodA[7] = np.genfromtxt('colours-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodA[8] = np.genfromtxt('colours-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodA[9] = np.genfromtxt('colours-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodA[10] = np.genfromtxt('colours-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodA[11] = np.genfromtxt('colours-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodA[12] = np.genfromtxt('colours-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #The following sets of data are required for BPASS FUV and NUV data.
    BmodB[0] = np.genfromtxt('ionizing-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodB[1] = np.genfromtxt('ionizing-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodB[2] = np.genfromtxt('ionizing-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodB[3] = np.genfromtxt('ionizing-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodB[4] = np.genfromtxt('ionizing-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodB[5] = np.genfromtxt('ionizing-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodB[6] = np.genfromtxt('ionizing-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodB[7] = np.genfromtxt('ionizing-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodB[8] = np.genfromtxt('ionizing-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodB[9] = np.genfromtxt('ionizing-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodB[10] = np.genfromtxt('ionizing-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodB[11] = np.genfromtxt('ionizing-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodB[12] = np.genfromtxt('ionizing-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodB[0] = np.genfromtxt('ionizing-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodB[1] = np.genfromtxt('ionizing-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodB[2] = np.genfromtxt('ionizing-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodB[3] = np.genfromtxt('ionizing-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodB[4] = np.genfromtxt('ionizing-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodB[5] = np.genfromtxt('ionizing-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodB[6] = np.genfromtxt('ionizing-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodB[7] = np.genfromtxt('ionizing-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodB[8] = np.genfromtxt('ionizing-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodB[9] = np.genfromtxt('ionizing-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodB[10] = np.genfromtxt('ionizing-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodB[11] = np.genfromtxt('ionizing-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodB[12] = np.genfromtxt('ionizing-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #Convert ionizing photon production rates into AB magnitudes for FUV and NUV data.
    Smodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(51):
            for k in range(23):
                if k == 0:
                    Smodns[i][j][k] = SmodB[i][j][0]
                    Bmodns[i][j][k] = BmodB[i][j][0]
                elif k == 1:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                elif k == 2:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                else:
                    Smodns[i][j][k] = SmodA[i][j][k - 1]
                    Bmodns[i][j][k] = BmodA[i][j][k - 1]
    
    Smod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(23):
            for k in range(51):
                if j == 0:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                elif (6 + 0.1*k) > 8.5 and k != 50:
                    Smod[i][k][j] = (Smodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                    Bmod[i][k][j] = (Bmodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                else:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
     
    #Store number of spectral measurements from input array.
    count = 0
    for i in range(len(mag)):
        if mag[i] != -99:
            count = count + 1
            
    count1 = 0
    #Observed magnitude and error array
    obs = [0 for i in range(count)]
    obserr = [0 for i in range(count)]
    
    #Corresponding index for model
    modindex = [0 for i in range(count)]
    
    for i in range(len(mag)):
        if mag[i] != -99:
            obs[i - count1] = mag[i]   
            obserr[i - count1] = err[i]
            modindex[i - count1] = i + 1
        else:
            count1 = count1 + 1
            
    #Find observed V magnitude.
    Vobs = -99
    
    for i in range(count):
        if modindex[i] == 5:
            Vobs = obs[i]
    
    #Return error if no Vobs is found in measurements.
    if (Vobs == -99):
        print('Error: No Vmag')
        return [0 for i in range(12)]
    
    #Find observed I magnitude.
    Iobs = -99
    
    for i in range(count):
        if modindex[i] == 7:
            Iobs = obs[i]
    
    #Return error if no Iobs is found in measurements.
    if (Iobs == -99):
        print('Error: No Imag')
        return [0 for i in range(12)]
    
    
    ##### AGE, Z AND DUST ESTIMATES #####
    
    #set up lists which store the age bin widths
    binwidth = [0 for x in range(51)]
    #get age binwidths from models
    for i in range(51):
        if i == 0:
            binwidth[i] = 10**6.05
        else:
            binwidth[i] = (10**(Smod[0][i][0]+0.05) - 10**(Smod[0][i][0]-0.05))
            
    #store metallicity values in a list
    z = [0.00001 , 0.0001 , 0.001 , 0.002 , 0.003 , 0.004 , 0.006 , 0.008 , 0.010 , 0.014 , 0.020 , 0.030 , 0.040]
    logz = [0 for i in range(13)]
    for i in range(13):
        logz[i] = math.log10(z[i])
    
    #Metallicity bin weighting factors
    totalzbinwidth = 0.040 + 0.5*(0.040-0.030)
    zbinwidth = [0 for i in range(13)]
    for i in range(13):
        if i == 0:
            zbinwidth[i] = (z[i] + 0.5*(z[i+1] - z[i]))
        elif i == 12:
            zbinwidth[i] = (z[i] - z[i-1])
        else:
            zbinwidth[i] = (0.5*(z[i] - z[i-1]) + 0.5*(z[i+1] - z[i]))
            
    #Extinction Array corresponding to spectral filters.
    ext = [2.6517 , 2.6331 , 
    1.5578462941114875, 1.3093650372780916, 1.00, 0.7614405115045431, 0.5929401215210357, 0.2880391530575151, 0.1859713631042207, 0.1155345186186997 ,
    1.5822202835340584, 1.21960961977842, 0.8785056211191205, 0.673375361689415, 0.4882034412598298 , 
    1.8252115 , 1.63853876 , 1.34967994 , 1.26766918 , 1.01183735 , 0.90721259 , 0.59576871]
            
    #Create lists to store all liklihoods over all ages and metallicities.
    SPtotal = [1 for x in range(51*13)]
    BPtotal = [1 for x in range(51*13)]
    
    #Numerators and denominators for average best-fit values.
    Snumage = 0
    Sdenage = 0
    Bnumage = 0
    Bdenage = 0
    
    SnumA = 0
    SdenA = 0
    BnumA = 0
    BdenA = 0
    
    Snumz = 0
    Sdenz = 0
    Bnumz = 0
    Bdenz = 0
    
    #Lists to store all possible V-band extinction values.
    sAV = [[0 for i in range(51)] for j in range(13)]
    bAV = [[0 for i in range(51)] for j in range(13)]
    
     #Loop through all 51 model ages and 13 model metallicities.
    for j in range(13):
        for k in range(51):
            
            #Temporary model SEDs to be compared to observed data
            Smodtemp = [Smod[j][k][l] for l in range(23)]
            Bmodtemp = [Bmod[j][k][l] for l in range(23)]
            
            #Add line of sight extinction to temporary model data
            for i in range(22):
                Smodtemp[i+1] = Smodtemp[i+1] + ext[i]*Amin
                Bmodtemp[i+1] = Bmodtemp[i+1] + ext[i]*Amin
            
            #Find ectinction required for (V-I) model colour to match that of the observed data
            sAVI = (mag[4] - mag[6]) - (Smodtemp[5] - Smodtemp[7])
            sAV[j][k] = sAVI/(ext[4] - ext[6])
            bAVI = (mag[4] - mag[6]) - (Bmodtemp[5] - Bmodtemp[7])
            bAV[j][k] = bAVI/(ext[4] - ext[6])
            
            #Exclude unphysical Av values
            if sAV[j][k] < 0:
                sAV[j][k] = 0
            if bAV[j][k] < 0:
                bAV[j][k] = 0
                
            #Add further dust to models, hence making (V-I) colours match between observed and temporary model SEDs.
            for i in range(22):
                Smodtemp[i+1] = Smodtemp[i+1] + ext[i]*sAV[j][k]
                Bmodtemp[i+1] = Bmodtemp[i+1] + ext[i]*bAV[j][k]
            
            
            #Difference in V-band magnitude between observed and model SEDs.
            Sdiff = Smodtemp[5] - Vobs
            Bdiff = Bmodtemp[5] - Vobs
            
            
            #Create lists containing model data for this specific age and metallicity
            Smodel = [0 for i in range(count)]
            Bmodel = [0 for i in range(count)]
            
            Smodelerr = [0 for i in range(count)]
            Bmodelerr = [0 for i in range(count)]
            
            #Scale model data so that V-band magitudes match that of the observed data.
            for i in range(count):
                Smodel[i] = Smodtemp[modindex[i]] - Sdiff
                Bmodel[i] = Bmodtemp[modindex[i]] - Bdiff
                
                #Calculate errors in Sin and Bin models.
                if k == 0:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                elif k == 50:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k - 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k - 1][modindex[i]])/2
                else:
                    Smodelerr[i] = abs(Smod[j][k - 1][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k - 1][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                
            '''Create list for relative liklihoods of model and observed data points matching (count - 2 as we do not use the 
            data point corresponding to V and I)'''
            Slike = [0 for i in range(count - 2)]
            Blike = [0 for i in range(count - 2)]
            
            
            #Switches for V-band and I-band magnitudes in order to skip it in the evaluation.
            switch = 0
            switch2 = 0
            for i in range(count):
                if modindex[i] == 5:
                    switch = 1
                elif modindex[i] == 7:
                    switch2 = 1
                else:
                    
                    #Total errors (observerd + model + dist. mod.) for Sin and Bin likelihood.
                    ser = math.sqrt(Smodelerr[i]**2 + obserr[i]**2)
                    ber = math.sqrt(Bmodelerr[i]**2 + obserr[i]**2)
                    
                    #Likelihood that Sin/Bin filter matching corrsponding observed value.
                    Slike[i - switch - switch2] = math.exp(-(Smodel[i] - obs[i])**2/(2*ser**2))
                    Blike[i - switch - switch2] = math.exp(-(Bmodel[i] - obs[i])**2/(2*ber**2))
                    
                    
            #Add probability of cluster surviving at a given age
            Page = 10**((6-Bmod[j][k][0])*0.1)
            
            #Store liklihood of model matching data
            for i in range(count - 2):
                SPtotal[51*j + k] = SPtotal[51*j + k]*Slike[i]
                BPtotal[51*j + k] = BPtotal[51*j + k]*Blike[i]
            
            #Calculate average best-fit parameters over all metallicities and ages
            
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                
                #Single-Star Age
                Snumage = Snumage + binwidth[k]*SPtotal[51*j + k]*Smod[j][k][0]*Page
                Sdenage = Sdenage + binwidth[k]*SPtotal[51*j + k]*Page
                #Binary Age
                Bnumage = Bnumage + binwidth[k]*BPtotal[51*j + k]*Bmod[j][k][0]*Page
                Bdenage = Bdenage + binwidth[k]*BPtotal[51*j + k]*Page
                
                #Single-Star A_V
                SnumA = SnumA + SPtotal[51*j + k]*sAV[j][k]
                SdenA = SdenA + SPtotal[51*j + k]
                #Binary A_V
                BnumA = BnumA + BPtotal[51*j + k]*bAV[j][k]
                BdenA = BdenA + BPtotal[51*j + k]
                
                #Single-Star Z
                Snumz = Snumz + zbinwidth[j]*SPtotal[51*j + k]*logz[j]
                Sdenz = Sdenz + zbinwidth[j]*SPtotal[51*j + k]
                #Binary Z
                Bnumz = Bnumz + zbinwidth[j]*BPtotal[51*j + k]*logz[j]
                Bdenz = Bdenz + zbinwidth[j]*BPtotal[51*j + k]
            
    #Avoid dividing by zero if models show poor match to data.
    if (Sdenage == 0) or (Bdenage == 0) or (SdenA == 0) or (BdenA == 0) or (Sdenz == 0) or (Bdenz == 0):
        return [0 for i in range(12)]
    
    #Single and binary Average Best-Fit log(age\yrs)
    Sage = Snumage/Sdenage
    Bage = Bnumage/Bdenage
    
    #Single and binary Average Best-Fit A_V
    SDUST = SnumA/SdenA
    BDUST = BnumA/BdenA
    
    #Single and binary Average Best-Fit log(Z)
    SZ = Snumz/Sdenz
    BZ = Bnumz/Bdenz 
    
    ##### Error in Evaluations: #####
    
    Snumage = 0
    Sdenage = 0
    Bnumage = 0
    Bdenage = 0
    
    SnumA = 0
    SdenA = 0
    BnumA = 0
    BdenA = 0
    
    Snumz = 0
    Sdenz = 0
    Bnumz = 0
    Bdenz = 0
    
    for j in range(13):
        for k in range(51):
            
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                Snumage = Snumage + binwidth[k]*SPtotal[51*j + k]*Page*(Smod[j][k][0] - Sage)**2
                Sdenage = Sdenage + binwidth[k]*SPtotal[51*j + k]*Page
                
                Bnumage = Bnumage + binwidth[k]*BPtotal[51*j + k]*Page*(Bmod[j][k][0] - Bage)**2
                Bdenage = Bdenage + binwidth[k]*BPtotal[51*j + k]*Page
                
                SnumA = SnumA + SPtotal[51*j + k]*(sAV[j][k] - SDUST)**2
                SdenA = SdenA + SPtotal[51*j + k]
                
                BnumA = BnumA + BPtotal[51*j + k]*(bAV[j][k] - BDUST)**2
                BdenA = BdenA + BPtotal[51*j + k]
                
                Snumz = Snumz + zbinwidth[j]*SPtotal[51*j + k]*(logz[j] - SZ)**2
                Sdenz = Sdenz + zbinwidth[j]*SPtotal[51*j + k]
                
                Bnumz = Bnumz + zbinwidth[j]*BPtotal[51*j + k]*(logz[j] - BZ)**2
                Bdenz = Bdenz + zbinwidth[j]*BPtotal[51*j + k]
            
    #Avoid dividing by zero if models show poor match to data.
    if (Sdenage == 0) or (Bdenage == 0) or (SdenA == 0) or (BdenA == 0) or (Sdenz == 0) or (Bdenz == 0):
        return [0 for i in range(12)]
    
    #error in <log(age/yrs)>
    eSage = Snumage/Sdenage
    eBage = Bnumage/Bdenage
    
    #error in A_V
    eSDUST = SnumA/SdenA
    eBDUST = BnumA/BdenA
    
    #error in <log(Z)>
    eSZ = Snumz/Sdenz
    eBZ = Bnumz/Bdenz
    
    #Return 1 x 12 output array containing estimates: 
    '''[Average best-fit Single log(age/yrs), associated uncertainty, Average best-fit 
    Binary log(age/yrs), associated uncertainty, Average best-fit Single log(Z), associated 
    uncertainty, Average best-fit Binary log(Z), associated uncertainty, Average best-fit 
    Single A_V, associated uncertainty, Average best-fit Binary A_V, associated uncertainty]'''
    return[Sage,eSage,Bage,eBage,SZ,eSZ,BZ,eBZ,SDUST+Amin,eSDUST,BDUST+Amin,eBDUST]

In [1]:
'''This function evaluates SED of a stellar cluster using a maximum likelihood 
estimation to produce an average best-fit Mass in conjunction with the absolute V-band extinction. 
Inputs are two 1 x 22 arrays, a 1 x 2 array and a float. 
The first 1 x 22 array contains the apparent magnitudes in the following spectral filters: 
mag = [FUV, NUV, U, B, V, R, I, J, H, K, u, g, r, i, z, f300w, f336w, f435w, f450w, f555w, f606w, f814w]. 
The second array, err, contains the associated errors within each spectral measurement. 
If data is unavailable for a particular filter then its corresponding array element is set to -99. 
All magnitudes are required to be given in the Vega system except for the FUV and 
NUV filters which are required to be in the AB system. The 1 x 2 array input specifies 
the distance modulus of the stellar cluster and the associated uncertainty, 
dmod = [Distance Modulus, Error in Measurement]. Finally, The fourth input, Amin, is a float 
specifying the minimum V-band extinction for the object. Output is a 1 x 8 array containing 
the average best-fit log(M/M_solar) and A_V values using the single-star and binary model sets:
[Average best-fit Single log(M/M_solar), associated uncertainty, Average best-fit 
Binary log(M/M_solar), associated uncertainty, Average best-fit Single A_V, 
associated uncertainty, Average best-fit Binary A_V, associated uncertainty]. 
Refer to associated dissertation for a full descripton of this code.'''

def Massfitv2(mag , err, Amin = None, dmod = None):
    
    #Give error message if no value for the distance modulus is given.
    if dmod is None:
        print('Error: No Distance Modulus')
        return[0 for i in range(8)]
    
    #Default minimum V-band extinction is zero.
    if Amin is None:
        Amin = 0
    
    import numpy as np
    import math
    
     #Give error message if inputs are in incorrect format.
    if len(mag) != 22 or len(err) != 22:
        print('Error: Inputs Have Incorrect Format')
        return [0 for i in range(8)]
    
    #Load BPASS v. 2.2 Single and Binary model data sets for imf135_300.
    BmodA = [[] for i in range(13)]
    SmodA = [[] for i in range(13)]
    BmodB = [[] for i in range(13)]
    SmodB = [[] for i in range(13)]

    BmodA[0] = np.genfromtxt('colours-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodA[1] = np.genfromtxt('colours-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodA[2] = np.genfromtxt('colours-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodA[3] = np.genfromtxt('colours-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodA[4] = np.genfromtxt('colours-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodA[5] = np.genfromtxt('colours-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodA[6] = np.genfromtxt('colours-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodA[7] = np.genfromtxt('colours-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodA[8] = np.genfromtxt('colours-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodA[9] = np.genfromtxt('colours-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodA[10] = np.genfromtxt('colours-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodA[11] = np.genfromtxt('colours-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodA[12] = np.genfromtxt('colours-bin-imf135_300.z040.dat', delimiter = '  ')

    #The following sets of data are required for BPASS FUV and NUV data.
    SmodA[0] = np.genfromtxt('colours-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodA[1] = np.genfromtxt('colours-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodA[2] = np.genfromtxt('colours-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodA[3] = np.genfromtxt('colours-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodA[4] = np.genfromtxt('colours-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodA[5] = np.genfromtxt('colours-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodA[6] = np.genfromtxt('colours-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodA[7] = np.genfromtxt('colours-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodA[8] = np.genfromtxt('colours-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodA[9] = np.genfromtxt('colours-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodA[10] = np.genfromtxt('colours-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodA[11] = np.genfromtxt('colours-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodA[12] = np.genfromtxt('colours-sin-imf135_300.z040.dat', delimiter = '  ')
    
    BmodB[0] = np.genfromtxt('ionizing-bin-imf135_300.zem5.dat', delimiter = '  ')
    BmodB[1] = np.genfromtxt('ionizing-bin-imf135_300.zem4.dat', delimiter = '  ')
    BmodB[2] = np.genfromtxt('ionizing-bin-imf135_300.z001.dat', delimiter = '  ')
    BmodB[3] = np.genfromtxt('ionizing-bin-imf135_300.z002.dat', delimiter = '  ')
    BmodB[4] = np.genfromtxt('ionizing-bin-imf135_300.z003.dat', delimiter = '  ')
    BmodB[5] = np.genfromtxt('ionizing-bin-imf135_300.z004.dat', delimiter = '  ')
    BmodB[6] = np.genfromtxt('ionizing-bin-imf135_300.z006.dat', delimiter = '  ')
    BmodB[7] = np.genfromtxt('ionizing-bin-imf135_300.z008.dat', delimiter = '  ')
    BmodB[8] = np.genfromtxt('ionizing-bin-imf135_300.z010.dat', delimiter = '  ')
    BmodB[9] = np.genfromtxt('ionizing-bin-imf135_300.z014.dat', delimiter = '  ')
    BmodB[10] = np.genfromtxt('ionizing-bin-imf135_300.z020.dat', delimiter = '  ')
    BmodB[11] = np.genfromtxt('ionizing-bin-imf135_300.z030.dat', delimiter = '  ')
    BmodB[12] = np.genfromtxt('ionizing-bin-imf135_300.z040.dat', delimiter = '  ')

    SmodB[0] = np.genfromtxt('ionizing-sin-imf135_300.zem5.dat', delimiter = '  ')
    SmodB[1] = np.genfromtxt('ionizing-sin-imf135_300.zem4.dat', delimiter = '  ')
    SmodB[2] = np.genfromtxt('ionizing-sin-imf135_300.z001.dat', delimiter = '  ')
    SmodB[3] = np.genfromtxt('ionizing-sin-imf135_300.z002.dat', delimiter = '  ')
    SmodB[4] = np.genfromtxt('ionizing-sin-imf135_300.z003.dat', delimiter = '  ')
    SmodB[5] = np.genfromtxt('ionizing-sin-imf135_300.z004.dat', delimiter = '  ')
    SmodB[6] = np.genfromtxt('ionizing-sin-imf135_300.z006.dat', delimiter = '  ')
    SmodB[7] = np.genfromtxt('ionizing-sin-imf135_300.z008.dat', delimiter = '  ')
    SmodB[8] = np.genfromtxt('ionizing-sin-imf135_300.z010.dat', delimiter = '  ')
    SmodB[9] = np.genfromtxt('ionizing-sin-imf135_300.z014.dat', delimiter = '  ')
    SmodB[10] = np.genfromtxt('ionizing-sin-imf135_300.z020.dat', delimiter = '  ')
    SmodB[11] = np.genfromtxt('ionizing-sin-imf135_300.z030.dat', delimiter = '  ')
    SmodB[12] = np.genfromtxt('ionizing-sin-imf135_300.z040.dat', delimiter = '  ')
    
    #Convert ionizing photon production rates into AB magnitudes for FUV and NUV data.
    Smodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmodns = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(51):
            for k in range(23):
                if k == 0:
                    Smodns[i][j][k] = SmodB[i][j][0]
                    Bmodns[i][j][k] = BmodB[i][j][0]
                elif k == 1:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][3]/((1.4*10**-15)*(4*math.pi*(3.086*10**19)**2))) + 18.82
                elif k == 2:
                    Smodns[i][j][k] = -2.5*math.log10(10**SmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                    Bmodns[i][j][k] = -2.5*math.log10(10**BmodB[i][j][4]/((2.06*10**-16)*(4*math.pi*(3.086*10**19)**2))) + 20.08
                else:
                    Smodns[i][j][k] = SmodA[i][j][k - 1]
                    Bmodns[i][j][k] = BmodA[i][j][k - 1]
    
    Smod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    Bmod = [[[0 for i in range(23)] for j in range(51)] for k in range(13)]
    
    for i in range(13):
        for j in range(23):
            for k in range(51):
                if j == 0:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
                elif (6 + 0.1*k) > 8.5 and k != 50:
                    Smod[i][k][j] = (Smodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                    Bmod[i][k][j] = (Bmodns[i][k][j] + Smodns[i][k - 1][j] + Smodns[i][k + 1][j])/3
                else:
                    Smod[i][k][j] = Smodns[i][k][j]
                    Bmod[i][k][j] = Bmodns[i][k][j]
     
    #Store number of spectral measurements from input array.
    count = 0
    for i in range(len(mag)):
        if mag[i] != -99:
            count = count + 1
            
    count1 = 0
    #Observed magnitude and error array
    obs = [0 for i in range(count)]
    obserr = [0 for i in range(count)]
    
    #Corresponding index for model
    modindex = [0 for i in range(count)]
    
    for i in range(len(mag)):
        if mag[i] != -99:
            obs[i - count1] = mag[i]   
            obserr[i - count1] = err[i]
            modindex[i - count1] = i + 1
        else:
            count1 = count1 + 1
            
    #Find observed V magnitude.
    Vobs = -99
    
    for i in range(count):
        if modindex[i] == 5:
            Vobs = obs[i]
    
    #Return error if no Vobs is found in measurements.
    if (Vobs == -99):
        print('Error: No Vmag')
        return [0 for i in range(8)]
    
    #Find observed I magnitude.
    Iobs = -99
    
    for i in range(count):
        if modindex[i] == 7:
            Iobs = obs[i]
    
    #Return error if no Iobs is found in measurements.
    if (Iobs == -99):
        print('Error: No Imag')
        return [0 for i in range(8)]
    
    #Extinction Array
    ext = [2.6517 , 2.6331 , 
    1.5578462941114875, 1.3093650372780916, 1.00, 0.7614405115045431, 0.5929401215210357, 0.2880391530575151, 0.1859713631042207, 0.1155345186186997 ,
    1.5822202835340584, 1.21960961977842, 0.8785056211191205, 0.673375361689415, 0.4882034412598298 , 
    1.8252115 , 1.63853876 , 1.34967994 , 1.26766918 , 1.01183735 , 0.90721259 , 0.59576871]
    
    ##### MASS ESTIMATES #####
      
    #Absolute Magnitudes.
    Mabs = [0 for i in range(count)]
    eMabs = [0 for i in range(count)]

    #Convert observed data from apparent to absolute magnitudes
    for i in range(count):
        Mabs[i] = obs[i] - dmod[0]
        eMabs[i] = math.sqrt(obserr[i]**2 + dmod[1]**2)

    #Create lists to store all liklihoods over all model ages and metallicities.
    SPtotal = [1 for x in range(51*13)]
    BPtotal = [1 for x in range(51*13)]
    
    #Array to store all possible masses.
    logSmasstotal = [0 for x in range(51*13)]
    logBmasstotal = [0 for x in range(51*13)]

    #Numerators and denominators for average best-fit values.
    Snummass = 0
    Sdenmass = 0
    Bnummass = 0
    Bdenmass = 0
    
    SnumA = 0
    SdenA = 0
    BnumA = 0
    BdenA = 0

    #Lists to store all possible V-band extinction values.
    sAV = [[0 for i in range(51)] for j in range(13)]
    bAV = [[0 for i in range(51)] for j in range(13)]
    
     #Loop through all 51 model ages and 13 model metallicities.
    for j in range(13):
        for k in range(51):
            
            #Temporary model SEDs to be compared to observed data
            Smodtemp = [Smod[j][k][l] for l in range(23)]
            Bmodtemp = [Bmod[j][k][l] for l in range(23)]
            
            #Add line of sight extinction to temporary model data
            for i in range(22):
                Smodtemp[i+1] = Smodtemp[i+1] + ext[i]*Amin
                Bmodtemp[i+1] = Bmodtemp[i+1] + ext[i]*Amin
            
            #Find ectinction required for (V-I) model colour to match that of the observed data
            sAVI = (mag[4] - mag[6]) - (Smodtemp[5] - Smodtemp[7])
            sAV[j][k] = sAVI/(ext[4] - ext[6])
            bAVI = (mag[4] - mag[6]) - (Bmodtemp[5] - Bmodtemp[7])
            bAV[j][k] = bAVI/(ext[4] - ext[6])
            
            #Exclude unphysical Av values
            if sAV[j][k] < 0:
                sAV[j][k] = 0
            if bAV[j][k] < 0:
                bAV[j][k] = 0
                
            #Add further dust to models, hence making (V-I) colours match between observed and temporary model SEDs.
            for i in range(22):
                Smodtemp[i+1] = Smodtemp[i+1] + ext[i]*sAV[j][k]
                Bmodtemp[i+1] = Bmodtemp[i+1] + ext[i]*bAV[j][k]
                
            #Difference in absolute V-band magnitudes between observed and model data (use this value to find the mass).
            Sdiff = Smodtemp[5] - (Vobs - dmod[0])
            Bdiff = Bmodtemp[5] - (Vobs - dmod[0])

            #Create lists containing model data for this specific age and metallicity
            Smodel = [0 for i in range(count)]
            Bmodel = [0 for i in range(count)]
            
            Smodelerr = [0 for i in range(count)]
            Bmodelerr = [0 for i in range(count)]

            #Scale model data so that V-band magitudes match that of the observed data.
            for i in range(count):
                Smodel[i] = Smodtemp[modindex[i]] - Sdiff
                Bmodel[i] = Bmodtemp[modindex[i]] - Bdiff
                
                #Calculate errors in Sin and Bin models.
                if k == 0:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2
                elif k == 50:
                    Smodelerr[i] = abs(Smod[j][k][modindex[i]] - Smod[j][k - 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k][modindex[i]] - Bmod[j][k - 1][modindex[i]])/2
                else:
                    Smodelerr[i] = abs(Smod[j][k - 1][modindex[i]] - Smod[j][k + 1][modindex[i]])/2
                    Bmodelerr[i] = abs(Bmod[j][k - 1][modindex[i]] - Bmod[j][k + 1][modindex[i]])/2

            '''Create list for relative liklihoods of model and observed data points matching (count - 1 as we do not use the 
            data point corresponding to V)'''
            Slike = [0 for i in range(count - 2)]
            Blike = [0 for i in range(count - 2)]

            #Switches for V-band and I-band magnitudes in order to skip it in the evaluation.
            switch = 0
            switch2 = 0
            for i in range(count):
                if modindex[i] == 5:
                    switch = 1
                elif modindex[i] == 7:
                    switch2 = 1
                else:

                    #Total errors (observerd + model + dist. mod.) for Sin and Bin likelihood.
                    ser = math.sqrt(Smodelerr[i]**2 + obserr[i]**2 + dmod[1]**2)
                    ber = math.sqrt(Bmodelerr[i]**2 + obserr[i]**2 + dmod[1]**2)
                    
                    #Likelihood that Sin/Bin filter matching corrsponding observed value.
                    Slike[i - switch - switch2] = math.exp(-(Smodel[i] - Mabs[i])**2/(2*ser**2))
                    Blike[i - switch - switch2] = math.exp(-(Bmodel[i] - Mabs[i])**2/(2*ber**2))

            #Store liklihood of model matching data
            for i in range(count - 2):
                SPtotal[51*j + k] = SPtotal[51*j + k]*Slike[i]
                BPtotal[51*j + k] = BPtotal[51*j + k]*Blike[i]

            #Calculate mass based on difference between observed absolute magnitude in V filter and V model
            logSmasstotal[51*j + k] = math.log10(10**6*10**((Sdiff + ext[4]*sAV[j][k])/2.5))
            logBmasstotal[51*j + k] = math.log10(10**6*10**((Bdiff + ext[4]*bAV[j][k])/2.5))

            #Calculate average best-fit parameters over all metallicities and ages
            
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                
                #Single-Star Mass
                Snummass = Snummass + SPtotal[51*j + k]*logSmasstotal[51*j + k]
                Sdenmass = Sdenmass + SPtotal[51*j + k]
                #Binary Mass
                Bnummass = Bnummass + BPtotal[51*j + k]*logBmasstotal[51*j + k]
                Bdenmass = Bdenmass + BPtotal[51*j + k]
                
                #Single-Star A_V
                SnumA = SnumA + SPtotal[51*j + k]*sAV[j][k]
                SdenA = SdenA + SPtotal[51*j + k]
                #Binary A_V
                BnumA = BnumA + BPtotal[51*j + k]*bAV[j][k]
                BdenA = BdenA + BPtotal[51*j + k]

    #Avoid dividing by zero if models show poor match to data.
    if (Sdenmass == 0) or (Bdenmass == 0) or (SdenA == 0) or (BdenA == 0):
        return [0 for i in range(8)]

    #Single and binary Average Best-Fit log(M/M_solar)
    Smass = Snummass/Sdenmass
    Bmass = Bnummass/Bdenmass
    
    #Single and binary Average Best-Fit A_V
    SDUST = SnumA/SdenA
    BDUST = BnumA/BdenA

    ##### Error in Evaluations: #####
    
    Snummass = 0
    Sdenmass = 0
    Bnummass = 0
    Bdenmass = 0
    
    SnumA = 0
    SdenA = 0
    BnumA = 0
    BdenA = 0

    for j in range(13):
        for k in range(51):
            
            #Exclude model SEDs older than those of the universe in evaluation.
            if Smod[j][k][0] <= 10.2 and Bmod[j][k][0] <= 10.2:
                Snummass = Snummass + SPtotal[51*j + k]*(logSmasstotal[51*j + k] - Smass)**2
                Sdenmass = Sdenmass + SPtotal[51*j + k]

                Bnummass = Bnummass + BPtotal[51*j + k]*(logBmasstotal[51*j + k] - Bmass)**2
                Bdenmass = Bdenmass + BPtotal[51*j + k]
                
                SnumA = SnumA + SPtotal[51*j + k]*(sAV[j][k] - SDUST)**2
                SdenA = SdenA + SPtotal[51*j + k]
                
                BnumA = BnumA + BPtotal[51*j + k]*(bAV[j][k] - BDUST)**2
                BdenA = BdenA + BPtotal[51*j + k]

    #Avoid dividing by zero if models show poor match to data.
    if (Sdenmass == 0) or (Bdenmass == 0):
        return [0 for i in range(8)]
    
    #error in <log(M/M_solar)>
    eSmass = Snummass/Sdenmass
    eBmass = Bnummass/Bdenmass
    
    #error in <A_V>
    eSDUST = SnumA/SdenA
    eBDUST = BnumA/BdenA

    #Return 1 x 8 output array containing estimates: 
    '''[Average best-fit Single log(M/M_solar), associated uncertainty, Average best-fit 
    Binary log(M/M_solar), associated uncertainty, Average best-fit Single A_V, 
    associated uncertainty, Average best-fit Binary A_V, associated uncertainty]'''
    return[Smass,eSmass,Bmass,eBmass,SDUST+Amin,eSDUST,BDUST+Amin,eBDUST]